In [ ]:
! pip install --upgrade tensorflow_hub
! pip install tensorflow-text

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
train_df = pd.read_json('train.json')
print(train_df.shape)
train_df

In [ ]:
train_df['sentiments'].value_counts()

In [ ]:
# Class count
count_good, count_bad = train_df['sentiments'].value_counts()

# Divide by class
train_df_good = train_df[train_df['sentiments'] == 1]
train_df_bad = train_df[train_df['sentiments'] == 0]


In [ ]:
count_good, count_bad

In [ ]:
train_df_bad_oversampled = train_df_bad.sample(count_good, replace=True)

train_df_balanced = pd.concat([train_df_good, train_df_bad_oversampled], axis = 0)
train_df_balanced.shape

In [ ]:
train_df_balanced['sentiments'].value_counts()

In [ ]:
train_df_balanced

In [ ]:
train_df_balanced['reviews']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_df_balanced['reviews'], train_df_balanced['sentiments'], test_size=0.20, random_state=0, stratify=train_df_balanced['sentiments'])

In [ ]:
x_train.head(4)

In [ ]:
bert_preprocess = hub.KerasLayer("<path>/bert_en_uncased_preprocess_3.tar")
bert_encoder = hub.KerasLayer("<path>/bert_en_uncased_L-12_H-768_A-12_4.tar")

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
# Bert Layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural Network Layers
l = tf.keras.layers.Dropout(0.1, name='dropout')(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(l)

# Construct Final Model
model = tf.keras.Model(inputs=[text_input], outputs=[l])

In [ ]:
model.summary()

In [ ]:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
model.fit(x_train, y_train, epochs=5)

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
y_predicted = model.predict(x_test)
y_predicted = y_predicted.flatten()

In [ ]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, y_predicted)
cm

In [ ]:
! pip install seaborn

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sn
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
print(classification_report(y_test, y_predicted))

import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(1, input_shape=(1,)))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')

print(K.eval(model.optimizer.lr))

model.optimizer.get_config()

test_df = pd.read_json('test.json')
test_df.head()
print(test_df.shape)

test_review_preprocessed = bert_preprocess(test_df['reviews'])

bert_encoder(test_review_preprocessed)['pooled_output']

submission = pd.DataFrame({
     "Review": list(test_df['reviews']),
     "Sentiments":list(test_predictions),
})
submission.to_csv("submission.csv", index=False)